<a href="https://colab.research.google.com/github/Saultr21/IA-Y-BIGDATA/blob/main/TAREA%20CASE/Python_WEB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask-ngrok

In [2]:
!pip install flask

In [3]:
!pip install pyngrok

In [4]:
!ngrok config add-authtoken 2rdMQ1YNewisuTrSgbgd7UUfoyj_2g8NwcJ5YZcqVpH6BXz1T

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
import os

os.makedirs('templates', exist_ok=True)
os.makedirs('static', exist_ok=True)
os.makedirs('generated_files', exist_ok=True)

In [6]:
from flask import Flask, render_template, request, jsonify, make_response
from pyngrok import ngrok
import xml.etree.ElementTree as ET

app = Flask(__name__)

# Almacén para las clases y relaciones
classes = []
relations = []

@app.route('/')
def index():
    return render_template('UML.html')

@app.route('/add_class', methods=['POST'])
def add_class():
    data = request.json
    class_name = data['name']
    new_class = {
        'name': class_name,
        'attributes': [],
        'methods': []
    }
    classes.append(new_class)
    return jsonify({'status': 'success'})

@app.route('/add_attribute', methods=['POST'])
def add_attribute():
    data = request.json
    class_name = data['class_name']
    attribute = f"{data['visibility']} {data['name']}:{data['type']}"
    for cls in classes:
        if cls['name'] == class_name:
            cls['attributes'].append(attribute)
            break
    return jsonify({'status': 'success'})

@app.route('/add_method', methods=['POST'])
def add_method():
    data = request.json
    class_name = data['class_name']
    method = f"{data['visibility']} {data['name']}():{data['type']}"
    for cls in classes:
        if cls['name'] == class_name:
            cls['methods'].append(method)
            break
    return jsonify({'status': 'success'})

@app.route('/add_relation', methods=['POST'])
def add_relation():
    data = request.json
    relation = {
        'from_class': data['from_class'],
        'to_class': data['to_class'],
        'type': data['type'],
        'from_multiplicity': data['from_multiplicity'],
        'to_multiplicity': data['to_multiplicity']
    }
    relations.append(relation)
    return jsonify({'status': 'success'})

@app.route('/download_xmi', methods=['GET'])
def download_xmi():
    xmi = ET.Element('XMI', {
        'xmi.version': '2.1',
        'xmlns:xmi': 'http://schema.omg.org/spec/XMI/2.1',
        'xmlns:uml': 'http://www.omg.org/spec/UML/20090901'
    })

    model = ET.SubElement(xmi, 'uml:Model', {
        'xmi:type': 'uml:Model',
        'name': 'UMLModel'
    })

    for cls in classes:
        class_element = ET.SubElement(model, 'packagedElement', {
            'xmi:type': 'uml:Class',
            'name': cls['name']
        })
        for attr in cls['attributes']:
            visibility, rest = attr.split(' ', 1)
            name, type_ = rest.split(':')
            ET.SubElement(class_element, 'ownedAttribute', {
                'visibility': visibility,
                'name': name.strip(),
                'type': type_.strip()
            })
        for meth in cls['methods']:
            visibility, rest = meth.split(' ', 1)
            name, type_ = rest.split(':')
            ET.SubElement(class_element, 'ownedOperation', {
                'visibility': visibility,
                'name': name.replace('()', '').strip(),
                'type': type_.strip()
            })

    for rel in relations:
        relation_type = {
            'herencia': 'Generalization',
            'composición': 'Composition',
            'agregación': 'Aggregation',
            'dependencia': 'Dependency',
            'asociaciónDireccional': 'DirectedAssociation'
        }.get(rel['type'], 'Association')

        relation_element = ET.SubElement(model, 'packagedElement', {
            'xmi:type': f'uml:{relation_type}',
            'memberEnd': f"{rel['from_class']} {rel['to_class']}"
        })

        if relation_type not in ['Generalization', 'Dependency']:
            ET.SubElement(relation_element, 'ownedEnd', {
                'type': rel['from_class'],
                'multiplicity': rel['from_multiplicity']
            })
            ET.SubElement(relation_element, 'ownedEnd', {
                'type': rel['to_class'],
                'multiplicity': rel['to_multiplicity']
            })

    xmi_str = ET.tostring(xmi, encoding='utf-8', xml_declaration=True).decode('utf-8')

    response = make_response(xmi_str)
    response.headers['Content-Disposition'] = 'attachment; filename=diagram.xmi'
    response.headers['Content-Type'] = 'application/xml'
    return response

if __name__ == '__main__':
  # Inicia el túnel
  public_url = ngrok.connect(5000)
  print(f"Servidor Flask en: {public_url}")

  app.run(port=5000)



Servidor Flask en: NgrokTunnel: "https://c0ef-34-16-214-23.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
